# **Information visualization class** 
#### *Data visualitation in an ice cream shop*

*Panel is an open-source Python library that lets you create custom interactive web apps 
and dashboards by connecting user-defined widgets to plots, images, tables, or text.: https://panel.holoviz.org/*

### 0a. Install and import libraries

In [435]:
#pip install hvplot 

In [436]:
#pip install panel

In [437]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')
#import hvplot as hv
import hvplot.pandas # For create interactive data frames
from IPython.display import Image # For Dashboard preview

In [438]:
# cache data to improve dashboard performance
if 'data' not in pn.state.cache.keys():
    helados_2020 = pd.read_csv("C:/Users/alexc/Desktop/Helados/2020modificadoV2.csv", delimiter=';')
    pn.state.cache['data'] = helados_2020.copy()
else: 
    helados_2020 = pn.state.cache['data']

### 0b. Load files

#### *- If you want to load from google drive:*

In [439]:
#from google.colab import drive 
#drive.mount('/content/drive') 

In [440]:
#path = "/content/drive/MyDrive/csv/2020modificado.csv"
#helados_2020 = pd.read_csv(path, delimiter=';')
#helados_2020.describe

#### *- If you want to load from GitHub:*

In [441]:
#helados_2020 = pd.read_csv("https://github.com/alexchavez1980/infovis/blob/main/2020modificadoV3.csv", delimiter=';')
#helados_2020.head(3)

#### *-I load it from local drive:*

In [442]:
helados_2020 = pd.read_csv("C:/Users/alexc/Desktop/Helados/2020modificadoV3.csv", delimiter=';')
helados_2020.head(3)

,succodigo,turno,caja,venta,comprobante,vtaoperacion,clinombre,vtaestado,vtafecha,franja_turno,...,sucursal,desde,hasta,sucdescrip,desdeturno,hastaturno,desdecomanda,hastacomanda,filtrosobreventa,cliente
0,Pilar III,1,1,0,NaN,VF,Consumidor Final,Anulado,3/12/2020 19:34,3,...,1,3/12/2020 00:00,31/12/2020 23:55,TODAS,NaN,NaN,NaN,NaN,NaN,1
1,Pilar III,1,1,1,NaN,VF,Consumidor Final,Normal,3/12/2020 22:20,3,...,1,3/12/2020 00:00,31/12/2020 23:55,TODAS,NaN,NaN,NaN,NaN,NaN,1
2,Pilar III,1,1,2,NaN,VF,Consumidor Final,Normal,3/12/2020 22:21,3,...,1,3/12/2020 00:00,31/12/2020 23:55,TODAS,NaN,NaN,NaN,NaN,NaN,1


### 0.c ...a bit of data preprocessing

In [443]:
# "Drop/Delete indicated columns"
helados_2020 = helados_2020.drop(columns=
                                 [
                                  'succodigo',
                                  #'turno',
                                  #'caja',
                                  'venta',
                                  'comprobante',
                                  #'vtaoperacion',
                                  'clinombre',
                                  'vtaestado',
                                  #'vtafecha',
                                  #'franja_turno',
                                  #'dia_venta',
                                  'usulogin',
                                  'condvtapos',
                                  #'delivery',
                                  #'articulo',
                                  #'descrip',
                                  #'precio',
                                  #'cant',
                                  #'total',
                                  'venta1',
                                  'sucursal',
                                  #'desde',
                                  #'hasta',
                                  'sucdescrip',
                                  'desdeturno',
                                  'hastaturno',
                                  'desdecomanda',
                                  'hastacomanda',
                                  'filtrosobreventa',
                                  'cliente'
                                  ]) 
#helados_2020 = helados_2020.fillna(0) # Replace Na with 0
#helados_2020 = data[data['subtitles'] != 0] # Delete all zero-rows.
#data = data.assign(column_number=data.subtitles.index) # Extract index and Create new column_number
helados_2020.head(3)

,turno,caja,vtaoperacion,vtafecha,franja_turno,dia_venta,delivery,articulo,descrip,descrip_alex,precio,cant,total,desde,hasta
0,1,1,VF,3/12/2020 19:34,3,3,No,186,Baño de Chocolate,Mini postres,10,1,10,3/12/2020 00:00,31/12/2020 23:55
1,1,1,VF,3/12/2020 22:20,3,3,No,31,Pal.Limon x 1,Palitos,30,1,30,3/12/2020 00:00,31/12/2020 23:55
2,1,1,VF,3/12/2020 22:21,3,3,No,31,Pal.Limon x 1,Palitos,30,1,30,3/12/2020 00:00,31/12/2020 23:55


In [444]:
# Make df Pipeline Interactive
i_helados_2020 = helados_2020.interactive()

### 1. Total sales over time by description (Product list)

In [445]:
# Define Panel widgets
Diciembre_slider = pn.widgets.IntSlider(name='Diciembre 2020. Día: ', start=1, end=30, step=1, value=3)
Diciembre_slider

IntSlider(end=30, name='Diciembre 2020. Día: ', start=1, value=3)

In [446]:
# Radio buttons for "total" measures
yaxis_total = pn.widgets.RadioButtonGroup(
    name='Eje Y', 
    options=['total', 
             'cant',
            ],
    button_type='success'
)

In [447]:
descripcion = ['Mini postres',
                'Palitos',
                'Bon Bon',
                'Tentación',
                'Top Cookie',
                'Helado por kilo',
                'Postres',
                'Cremoso',
                'Familiar',
                'Almendrado',
                'Comidas rápidas',
                'Cafetería',
                'Cups',
                'Smoothie',
                ]
total_pipeline = (
    i_helados_2020[
        (i_helados_2020.dia_venta <= Diciembre_slider) &
        (i_helados_2020.descrip_alex.isin(descripcion))
    ]
    .groupby(['descrip_alex', 'dia_venta'])[yaxis_total].sum()
    .round(2) # Two decimals
    .to_frame()
    .reset_index()
    .sort_values(by='dia_venta')  
    .reset_index(drop=True)
)

In [448]:
total_pipeline.head()

### 1a. Total sales over time by description plot

In [449]:
total_plot = total_pipeline.hvplot(x = 'dia_venta', 
                                   by='descrip_alex', 
                                   y=yaxis_total,
                                   line_width=2, 
                                   title="Ventas a lo largo del mes",
                                   kind='line',
                                   legend='bottom')
total_plot

### 1b. Total sales over time by description table

In [450]:
total_table = total_pipeline.pipe(pn.widgets.Tabulator, 
                                  pagination='remote', 
                                  page_size = 10, 
                                  #sizing_mode='stretch_width',
                                  layout='fit_data_stretch', 
                                  header_align='center', 
                                  #title="Alex",
                                  text_align='center') 
total_table

### 1c. Total sales vs. quantity for products scatterplot

In [451]:
#First, I built the pipeline.
total_vs_cantidad_scatterplot_pipeline = (
    i_helados_2020[
        (i_helados_2020.dia_venta == Diciembre_slider) &
        (i_helados_2020.descrip_alex.isin(descripcion))
    ]
    .groupby(['descrip_alex', 'dia_venta', 'cant'])['total'].sum()
    .round(2) # Two decimals
    .to_frame()
    .reset_index()
    .sort_values(by='dia_venta')  
    .reset_index(drop=True)
)

In [452]:
total_vs_cantidad_scatterplot_pipeline.head(3)

In [453]:
#And finally, I built the scatterplot
total_vs_cantidad_scatterplot = total_vs_cantidad_scatterplot_pipeline.hvplot(x='cant', 
                                                                y='total', 
                                                                by='descrip_alex', 
                                                                size=80, 
                                                                kind="scatter", 
                                                                alpha=0.7,
                                                                legend='right',
                                                                height=500, 
                                                                width=1000,
                                                                title="Cantidades y total acumulado de productos",
                                                                )
total_vs_cantidad_scatterplot

### 1d. Bar chart "Total & Cant daily" sales by descripcion

In [454]:
yaxis_total_barra = pn.widgets.RadioButtonGroup(
    name='Eje Y', 
    options=['total', 
             'cant'
            ], 
    button_type='success'
)

solo_dos_productos = ['Helado por kilo', 'Comidas rápidas', 'Cafetería']

total_source_bar_pipeline = (
    i_helados_2020[
               (i_helados_2020.dia_venta == Diciembre_slider) &
               (i_helados_2020.descrip_alex.isin(solo_dos_productos))       
    ]
    .groupby(['dia_venta', 'descrip_alex'])[yaxis_total_barra].sum()
    .to_frame()
    .round(0) # Two decimals
    .reset_index()
    .sort_values(by='dia_venta')  
    .reset_index(drop=True)
)

In [455]:
total_bar_plot = total_source_bar_pipeline.hvplot(kind='bar', 
                                                     x='descrip_alex', 
                                                     y=yaxis_total_barra, 
                                                     #y='descrip_alex', 
                                                     #x=yaxis_total_barra, 
                                                     title='Productos a monitorear diariamente')
total_bar_plot

### 2. Dashboard 

In [456]:
#Image(filename='C:/Users/alexc/Desktop/Helados/Tablero.jpg')

In [457]:
#Layout using Template
template = pn.template.FastListTemplate(
    title='Heladería de la esquina', 
#    sidebar=[
#             pn.pane.Markdown("# Ventas y movimientos"), 
#             pn.pane.Markdown("#### Tablero de control de los datos mas relevantes."), 
#             pn.pane.PNG('C:/Users/alexc/Desktop/Helados/Icecream.png', sizing_mode='scale_both'),
#             Diciembre_slider
#            ],
    main=[
          Diciembre_slider,
          pn.Row(pn.Column(yaxis_total, 
                           total_plot.panel(width=800,height=500), 
                           margin=(0,25), 
                           width=1000, 
                           height=550, scroll=True)), 
          pn.Row(pn.Column(yaxis_total, 
                           total_table.panel(width=500), 
                           margin=(0,25))),                            
          pn.Row(pn.Column(total_vs_cantidad_scatterplot.panel(width=800,height=500), margin=(0,25))), 
          pn.Row(pn.Column(yaxis_total_barra,total_bar_plot.panel(width=1000)))],
          accent_base_color="#88d8b0",
          header_background="#88d8b0",
)
template.show()
template.servable();

Launching server at http://localhost:53220


#### *Now, I´m going to the terminal and use the command panel serve, followed by InfoVis_Helados.ipynb*